In [1]:
#imports
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.layers import Dense,SpatialDropout1D
import contractions
import re 
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
# initializing Stop words libraries
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.tokenize import word_tokenize


RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

Init Plugin
Init Graph Optimizer
Init Kernel


[nltk_data] Downloading package punkt to /Users/olohireme/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/olohireme/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/olohireme/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def clean(desc):
    desc = contractions.fix(desc)
    desc = re.sub("[!@.$\'\'':()]", "", desc)
    return desc

In [3]:
def tokenize_and_tag(desc):
    tokens = nltk.word_tokenize(desc.lower())
    filtered_tokens = [w for w in tokens if not w in stop_words]
    tagged = nltk.pos_tag(filtered_tokens)
    return tagged

In [4]:
def extract_POS(tagged):
    #pattern 1
    grammar1 = ('''Noun Phrases: {<DT>?<JJ>*<NN|NNS|NNP>+}''')
    chunkParser = nltk.RegexpParser(grammar1)
    tree1 = chunkParser.parse(tagged)

    # typical noun phrase pattern appending to be concatted later
    g1_chunks = []
    for subtree in tree1.subtrees(filter=lambda t: t.label() == 'Noun Phrases'):
        g1_chunks.append(subtree)
    
    #pattern 2
    grammar2 = ('''NP2: {<IN>?<JJ|NN>*<NNS|NN>} ''')
    chunkParser = nltk.RegexpParser(grammar2)
    tree2 = chunkParser.parse(tagged)

    # variation of a noun phrase pattern to be pickled for later analyses
    g2_chunks = []
    for subtree in tree2.subtrees(filter=lambda t: t.label() == 'NP2'):
        g2_chunks.append(subtree)
        
    #pattern 3
    grammar3 = (''' VS: {<VBG|VBZ|VBP|VBD|VB|VBN><NNS|NN>*}''')
    chunkParser = nltk.RegexpParser(grammar3)
    tree3 = chunkParser.parse(tagged)

    # verb-noun pattern appending to be concatted later
    g3_chunks = []
    for subtree in tree3.subtrees(filter=lambda t: t.label() == 'VS'):
        g3_chunks.append(subtree)
        
        
    # pattern 4
    # any number of a singular or plural noun followed by a comma followed by the same noun, noun, noun pattern
    grammar4 = ('''Commas: {<NN|NNS>*<,><NN|NNS>*<,><NN|NNS>*} ''')
    chunkParser = nltk.RegexpParser(grammar4)
    tree4 = chunkParser.parse(tagged)

    # common pattern of listing skills appending to be concatted later
    g4_chunks = []
    for subtree in tree4.subtrees(filter=lambda t: t.label() == 'Commas'):
        g4_chunks.append(subtree)
        
    return g1_chunks, g2_chunks, g3_chunks, g4_chunks


In [5]:
def training_set(chunks):
    '''creates a dataframe that easily parsed with the chunks data '''
    df = pd.DataFrame(chunks)    
    df.fillna('X', inplace = True)
    
    train = []
    for row in df.values:
        phrase = ''
        for tup in row:
            # needs a space at the end for seperation
            phrase += tup[0] + ' '
        phrase = ''.join(phrase)
        # could use padding tages but encoder method will provide during 
        # tokenizing/embeddings; X can replace paddding for now
        train.append( phrase.replace('X', '').strip())

    df['phrase'] = train

    #returns 50% of each dataframe to be used if you want to improve execution time
    # return df.phrase.sample(frac = 0.5)
    # Update: only do 50% if running on excel
    return df.phrase

def strip_commas(df):
    '''create new series of individual n-grams'''
    grams = []
    for sen in df:
        sent = sen.split(',')
        for word in sent:
            grams.append(word)
    return pd.Series(grams)



In [6]:
def generate_phrases(desc):
    tagged = tokenize_and_tag(desc)
    g1_chunks, g2_chunks, g3_chunks, g4_chunks = extract_POS(tagged)
    c = training_set(g4_chunks)       
    separated_chunks4 = strip_commas(c)
    phrases = pd.concat([training_set(g1_chunks),
                          training_set(g2_chunks), 
                          training_set(g3_chunks),
                          separated_chunks4], 
                            ignore_index = True )
    return phrases

In [7]:
"""Creates corpus from feature column, which is a pandas series"""
def create_corpus(df):
    corpus=[]
    for phrase in tqdm(df):
        words=[word.lower() for word in word_tokenize(phrase) if(word.isalpha()==1)]
        corpus.append(words)
    return corpus

In [8]:
"""Create padded sequences of equal lenght as input to LSTM"""
def create_padded_inputs(corpus):
    MAX_LEN=20
    tokenizer_obj=Tokenizer()
    tokenizer_obj.fit_on_texts(corpus)
    sequences=tokenizer_obj.texts_to_sequences(corpus)

    phrase_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')
    return phrase_pad

In [9]:
def get_predictions(desc):
    #clean
    desc = clean(desc)
    #load model
    model = tf.keras.models.load_model('models/lstm_skill_extractor.h5')
    #tokenize and convert to phrases
    phrases = generate_phrases(desc)
    #preprocess unseen data
    corpus=create_corpus(phrases)
    corpus_pad = create_padded_inputs(corpus)
    #get predicted classes
    predictions = (model.predict(corpus_pad) > 0.65).astype('int32')
    #return predicted skills as list
    out = pd.DataFrame({'Phrase':phrases, 'Class':predictions.ravel()})
    skills = out.loc[out['Class'] == 1]
    return skills['Phrase'].tolist()

In [10]:
#test with predictions on ketter's repo
def get_predictions_excel(filename):
    """description column must be titled Job Desc"""
    df = pd.read_csv(filename)
    df['Extracted skills'] = df['Job Description'].apply(lambda x: get_predictions(x))
    
    return df.to_csv('extracted.csv')

    #throw error if column name does not exist or file format is wrong

In [11]:
#test get predictions
desc = """At Kindred, we are on a mission to build human-like intelligence in machines.

 

Since 2014, we've been paving the way for a world filled with more powerful and helpful AI systems. We bring together reinforcement learning, machine learning, and remote human guidance to create intelligent robots that solve real-world problems alongside humans in complex, changing environments like today's supply chain.

 

The company is growing rapidly and we are searching for an individual for a highly-impactful role, has great attention to detail, is a proactive self-starter and interacts easily with all levels of a high-performance team.


About this role:

As an Intermediate to Senior Machine Learning Engineer, you are a member of our Machine Learning Engineering & SW Platform team. You will design and develop back-end infrastructure to support ML efforts & product features. You will be contributing to design and architecture of complex systems that integrate various hardware components. You will bring up and balance considerations for immediate and future needs for scalability, reliability, security, and robustness. You will closely collaborate with researchers, engineers, and product managers in Toronto and other offices. In this role, you will be part of an on-call rotation in order to service our customers across North America.

 


What you bring: 

Familiar with a variety of Machine Learning tools and packages, understand how to properly set up experiments
Good understanding of the mathematical and statistical foundations of ML algorithms
Experience with various big data tools and cloud technologies that enable training algorithms at scale
Actively stay on top of applied machine learning and big data technologies
Desire to investigate potential algorithmic solutions for a variety of machine learning problems
Able to tackle varied problems, balancing a long-term mission with short-term requirements

Technical Skills: 

Significant development experience in Python on Linux
Reliable production of quality software within time and resource constraints
Complete familiarity with modern software development processes such as design documentation, code reviews, CI/CD, testing, project management workflow, and source control conventions
Excellent analytical, problem-solving, communication, and organization skills
Bachelor's degree in Computer Science or equivalent experience

Bonus Qualifications:

Proficiency with Golang
Experience with deep learning packages such as Tensorflow or PyTorch
Background in robotics, computer vision, and deep learning"""
get_predictions(desc)

Metal device set to: Apple M1


2021-08-26 23:20:10.303034: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-08-26 23:20:10.303157: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
100%|██████████████████████████████████████| 198/198 [00:00<00:00, 23560.16it/s]
2021-08-26 23:20:10.728942: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-26 23:20:10.729402: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-08-26 23:20:10.802645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for de

['machine learning',
 'member machine',
 'various hardware components',
 'product managers',
 'familiar variety machine learning tools packages',
 'various big data tools',
 'big data technologies',
 'investigate potential algorithmic solutions variety machine',
 'technical skills',
 'significant development experience python linux',
 'code reviews',
 'organization skills bachelors',
 'computer science',
 'equivalent experience bonus qualifications proficiency',
 'pytorch background robotics',
 'computer vision',
 'machine learning',
 'remote human guidance create intelligent robots',
 'high-performance team role intermediate senior machine',
 'member machine',
 'various hardware components',
 'product managers',
 'across north america bring familiar variety machine learning tools',
 'algorithms experience various big data tools',
 'variety machine',
 'technical skills',
 'significant development experience python linux reliable production quality software',
 'code reviews',
 'computer

## Rule-Based Model

In [22]:
np.__version__

'1.19.5'

In [21]:
import spacy
from spacy.util import filter_spans
from spacy.tokens import Span
from spacy.matcher import Matcher

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
def get_left_span(tok, label='', include=True):
    offset = 1 if include else 0
    idx = tok.i
    while idx > tok.left_edge.i:
        if tok.doc[idx - 1].pos_ in ('NOUN', 'PROPN', 'ADJ', 'X'):
            idx -= 1
        else:
            break
    return label, idx, tok.i+offset

In [ ]:
def get_conjugations(tok):
    new = [tok]
    while new:
        tok = new.pop()
        yield tok
        for child in tok.children:
            if child.dep_ == 'conj':
                new.append(child)

In [ ]:
EXP_TERMS = ['experience']
def extract_adp_conj_experience(doc, label='EXPERIENCE'):
    for tok in doc:
        if tok.lower_ in EXP_TERMS:
            for child in tok.rights:
                if child.dep_ == 'prep':
                    for obj in child.children:
                        if obj.dep_ == 'pobj':
                            for conj in get_conjugations(obj):
                                yield get_left_span(conj, label)

In [ ]:
def get_extractions(examples, *extractors):
    # Could use context instead of enumerate
    doc = nlp(examples, disable=['ner'])
    for ent in filter_spans([Span(doc, start, end, label) for extractor in extractors for label, start, end in extractor(doc)]):
        sent = ent.root.sent
        yield ent.text

In [ ]:
def list_skills(examples, *extractors):
    return list(get_extractions(examples, *extractors))

In [ ]:
print(list_skills(desc, extract_adp_conj_experience))